In [1]:
"""
author: gowthamkuntumalla

Title: Quantitative Analysis of Stock Prices for Similarity Grouping using Technical Indicator

# Quantitative Analysis: https://www.investopedia.com/terms/q/quantitativeanalysis.asp
# Technical Indicator: https://www.investopedia.com/terms/t/technicalindicator.asp

Statistical Learning Techniques are employed for clustering and such.
"""

import pandas as pd
import numpy as np
idx = pd.IndexSlice
from pathlib import Path

#plotting
import matplotlib.pyplot as plt
from bokeh.plotting import figure,show
from bokeh.models.sources import ColumnDataSource
from bokeh.models import HoverTool, DatetimeTickFormatter

# Features

There are two basic types of technical indicators:

**Overlays**: Technical indicators that use the same scale as prices are plotted over the top of the prices on a stock chart. Examples include moving averages and Bollinger Bands. <br>
**Oscillators**: Technical indicators that oscillate between a local minimum and maximum are plotted above or below a price chart. Examples include the stochastic oscillator, MACD or RSI.

## 1. Bollinger Bandwidth

In [2]:
def bollinger_bandwidth(m = 2, n = 20,col_name = 'bollinger_bandwidth'):
    """  
    Bollinger Bandwidth 
    
    https://www.investopedia.com/terms/b/bollingerbands.asp
    
    """
    # Bollinger Bandwidth = ((Upper Band - Lower Band) / Middle Band) * 100 
    # m = num of standard devs (typically 2)
    # n = No. days in smoothening period (typically 20)
    
    
    def MA(tp,n):
        """Moving Average"""
        return tp.rolling(n).mean()
    
    def TP(stock):
        """Typical Price"""
        return (stock['high']+stock['low']+stock['close'])/3 
    
    def stdev(tp,n):
        return tp.rolling(n).std()
        
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        stock_tp = TP(stock)
        middle_band = MA(stock_tp,n) # Middle Bollinger Band
        a = 2*m*stdev(stock_tp,n)/ middle_band 
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])

## 2. Coppock Curve

In [3]:
def coppock_curve(m = 10, n1= 14, n2 = 11, price_type = 'close',col_name = 'coppock_curve'):    
    """ 
    Coppock Curve
    
    https://www.investopedia.com/terms/c/coppockcurve.asp
    
    Using default price columns as 'close'
    """
    # 10-period WMA of (14-period RoC + 11-period RoC)
    # ROC = [(Close - Close n periods ago) / (Close n periods ago)] * 100 
    
    def RoC(stock_price,n): 
        """Rate of change"""
        return 100*stock[price_type].pct_change(periods = n)
        
    
    def WMA(m,roc_n1,roc_n2):
        """Weighted Moving Average"""  
        weights = np.arange(1,m+1) #this creates an array with integers 1 to 10 included
        return (roc_n1 + roc_n2).rolling(m).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        
        roc_n1 = RoC(stock,n1)
        roc_n2 = RoC(stock,n2)
        
        a = np.round(WMA(m,roc_n1,roc_n2),decimals = 3)/1.0
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
        
    

## 3. Correlation Coefficient

In [4]:
def corr_coef():
    """
    Correlation Coefficient
    
    """
    
    pass

## 4. MACD

In [5]:
def macd(nfast = 12 , nslow =26 ,m = 9,price_type = 'close',col_name = 'macd'):
    """
    Moving Average Convergence Divergence
    
    https://www.investopedia.com/terms/m/macd.asp
    
    Note: difference between macd and signal is stored for convenience
    """
    # macd = 12-Period EMA − 26-Period EMA
    # signal = 9 period ema of macd
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        
        macd = stock[price_type].ewm(span = nfast).mean()-stock[price_type].ewm(span = nslow).mean()
        signal = macd.ewm(span = m).mean()
        a = np.round(macd - signal,5)
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
        

## 5. Market Facilitation Index

In [6]:
def market_fac_ind(col_name = 'market_fac_ind'):
    """
    Market Facilitation Index
    
    https://en.wikipedia.org/wiki/Market_facilitation_index
    https://www.tradingview.com/script/trUemla9-Market-Facilitation-Index-MFI/
    
    Use the indicator to see if the market is trending
    """
    # (High - Low) / Volume
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        a = (stock['high'] - stock['low'])/stock['volume']
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
        
    
    

## 6. Momentum Indicator

In [7]:
def momentum_indic(n = 14,col_name = 'momentum_indic', price_type = 'close'):
    """
    Momentum Indicator
    
    https://commodity.com/technical-analysis/momentum/
    
    """
    # (Price today - Price n periods ago) x 100
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        
        a = stock[price_type] - stock[price_type].shift(periods = n)
        
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
        

## 7. Positive Volume Index

In [8]:
def pvi(col_name = 'pvi', price_type = 'close', ppvi = 100):
    """
    Positive Volume Index
    
    https://www.investopedia.com/terms/p/pvi.asp
    
    Implementation is slow. It may be improved!
    """
    # pvi = ppvi + (tcp-ycp)/ycp * ppvi
    # if tvol < yvol :then pvi = ppvi
    # if no starting value :then ppvi  = 100 # starting value arbritrary
    
    def func(stock,ppvi):
        a = pd.Series(stock.index)       
        a.iloc[0] = ppvi        
        for i in range(1, len(a)):            
            if stock['volume'].iloc[i] > stock['volume'].iloc[i-1]:
                pvi = ppvi * (1+ (stock[price_type].iloc[i]-stock[price_type].iloc[i-1])/stock[price_type].iloc[i-1])
                a.iloc[i] = pvi
                ppvi = pvi
            else:
                a.iloc[i] = ppvi
        return a
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        a = func(stock,ppvi)
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])



## 8. Relative Volatility Index

In [9]:
def rvi(n = 10, m = 14,col_name = 'rvi'):
    """
    Relative Volatility Index
    
    https://user42.tuxfamily.org/chart/manual/Relative-Volatility-Index.html
    
    """
    # n = stdev periods
    # m = EMA periods
    
    def func(stock,n,m,price_type):       
        stdev = stock[price_type].rolling(n).std()
        mask = stock[price_type] > stock[price_type].shift(periods = 1)
        
        numer = stdev.where(mask,other = 0).ewm(span = m, min_periods=m,adjust=False).mean()
        denom = stdev.ewm(span = m, min_periods=m,adjust=False).mean()
        
        rviOrig =   100 * (numer)/(denom)        
        return rviOrig
        
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        
        a = np.round(0.5*(func(stock,n,m,'high') + func(stock,n,m,'low')),3)

        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])


## 9. Standard Deviation

In [10]:
def st_dev(n = 10,col_name = 'st_dev'):
    """
    Standard Deviation
    
    """
    # n = no of days to calculate standard deviation

    def TP(stock):
        """Typical Price"""
        return (stock['high']+stock['low']+stock['close'])/3 
    
    def stdev(tp,n):
        """rolling standard deviation"""
        return tp.rolling(n).std()
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        stock_tp = TP(stock)
        a = stdev(stock_tp,n)
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
    

## 10. Stochastic Momentum Index

In [11]:
def smi(col_name = 'smi',n = 14, m = 3):
    """
    Stochastic Momentum Index
    
    https://www.fmlabs.com/reference/default.htm?url=SMI.htm
    https://tradingqna.com/t/need-formula-for-stochastic-momentum-index-indicator/43201
    """
    # n = period 1 for calculating centre
    # m = period 2 for EMA
    
    def func(stock,n,m):
        centre = 0.5 * (stock['high'].rolling(n).max() + stock['low'].rolling(n).min()) 
        H = stock['close'] - centre        
        cm = H.ewm(span = m).mean().ewm(span = m).mean()
        
        HL = stock['high'].rolling(n).max() - stock['low'].rolling(n).min()
        hl = HL.ewm(span = m).mean().ewm(span = m).mean()
        
        SMI = 100 * cm/(hl/2)
        return SMI
        
    
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        a = np.round(func(stock, n,m),3)
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
    


## 11. Stochastics

In [12]:
def stochastics(col_name = 'stochastics', n = 14,price_type = 'close'):
    """
    Stochastics (Stochastic Oscillator)
    
    https://www.investopedia.com/terms/s/stochasticoscillator.asp
    
    """
    # %K=(C- H14)/(H14−L14)×100
    
     
    for tick in tickers: 
        stock = stocks_df.loc[tick]
        
        a = 100 * (stock[price_type] - stock[price_type].rolling(n).min())\
                    /(stock[price_type].rolling(n).max() - stock[price_type].rolling(n).min())
        
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
    

## 12. Typical Price

In [13]:
def typical_price(col_name= 'typical_price'):
    """
    Typical Price
    
    https://en.wikipedia.org/wiki/Typical_price
    
    """    
    # H+L+C/3
    for tick in tickers:
        stock = stocks_df.loc[tick]
        a = (stock['high']+stock['low']+stock['close'])/3     
        a.rename(None,inplace=True)
        stocks_df.loc[tick,col_name] = pd.DataFrame(a,columns = [col_name])
        

# Import Data

In [14]:
dirname = 'sandp500'
csv_file = 'all_stocks_5yr.csv'
filename = Path(dirname, csv_file)
stocks_df = pd.read_csv(filename)
stocks_df.set_index(['Name','date'],inplace = True) # contains all stocks time series data
stocks_df.head()

open   high    low  close    volume
Name date                                           
AAL  2/8/2013   15.07  15.12  14.63  14.75   8407500
     2/11/2013  14.89  15.01  14.26  14.46   8882000
     2/12/2013  14.45  14.51  14.10  14.27   8126000
     2/13/2013  14.30  14.94  14.25  14.66  10259500
     2/14/2013  14.94  14.96  13.16  13.99  31879900

# Calling Features

In [15]:
## Names of all feature functions
feature_funcs = [bollinger_bandwidth,coppock_curve,corr_coef,macd,\
                 market_fac_ind,momentum_indic,pvi,rvi,st_dev,smi,stochastics,typical_price]

## Stock Tickers. Ex: 'AAPL', 'MSFT' ...
tickers = stocks_df.index.get_level_values(0).unique().tolist() # len(tickers) == 505

for func in feature_funcs:
    func()

    """Note:
    1. Corr_coeff does nothing here
    2. pvi is slow
    3. It may take 5 min to run all functions
    4. There are 11 feature columns in addition to the original 5 data columns""" 

In [16]:
stocks_df.describe()

,open,high,low,close,volume,bollinger_bandwidth,coppock_curve,macd,market_fac_ind,momentum_indic,rvi,st_dev,smi,stochastics,typical_price
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05,609352.000000,607425.000000,619040.000000,6.190320e+05,611970.000000,607917.000000,614442.000000,612462.000000,612475.000000,619032.000000
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06,0.094218,1.447813,0.003326,1.421292e-06,0.621775,52.682540,1.308930,11.756346,56.053871,83.025869
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06,0.064830,9.413234,0.631664,5.107095e-06,6.992943,13.191995,2.145651,52.050636,36.849452,97.365489
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00,0.006689,-127.764000,-30.106160,-1.311152e-07,-296.150000,1.726000,0.011387,-366.819000,0.000000,1.643333
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06,0.053378,-3.597000,-0.161240,1.848395e-07,-1.310000,44.077000,0.466277,-33.753750,20.606061,40.240000
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06,0.078014,1.473000,0.001800,4.795491e-07,0.420000,52.712000,0.808583,17.454000,61.500000,62.599933
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06,0.115608,6.525000,0.167210,1.214260e-06,2.330000,61.408000,1.432583,58.804000,93.368048,94.370000
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08,2.138135,239.239000,36.649300,1.227937e-03,304.630000,97.838000,122.301219,97.961000,100.000000,2050.700000


# Appendix - Bokeh Interactive Data Visualization

In [17]:
#dirname = 'sandp500/individual_stocks_5yr'
# filename = 'AAPL_data'
# suffix = '.csv'

# filename = Path(dirname, filename).with_suffix(suffix)
# aapl_df = pd.read_csv(filename)
# aapl_df.date = pd.to_datetime(aapl_df.date,infer_datetime_format=True);


## Sample Bokeh Code for Interactive Visualization
# plot = figure(x_axis_label = 'Time', y_axis_label = 'Price',x_axis_type = 'datetime',title = 'Closing Price of AAPL')
# aapl = ColumnDataSource(aapl_df)
# plot.circle('date','close',source = aapl)
# # plot.xaxis[0].formatter = DatetimeTickFormatter(days='%m/%d')
# plot.add_tools(HoverTool(tooltips= [("Dates","@date"), ("Close Prices","@close")]))
# show(plot)